In [4]:
!pip install ultralytics opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 100.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
from pathlib import Path
from shutil import copy2

# 設定原始資料夾與目標資料夾
orig_dir = Path("/kaggle/input/raw-dataset/CVPDL_hw2/CVPDL_hw2/train")  # 原始資料夾
dataset_dir = Path("/kaggle/working")
images_train_dir = dataset_dir / "dataset/images/train"
images_val_dir = dataset_dir / "dataset/images/val"
labels_train_dir = dataset_dir / "dataset/labels/train"
labels_val_dir = dataset_dir / "dataset/labels/val"

# convert to yolo

In [2]:
import os
import numpy as np
import cv2
import random
# 建立資料夾
for d in [images_train_dir, images_val_dir, labels_train_dir, labels_val_dir]:
    d.mkdir(parents=True, exist_ok=True)

# 假設你有一個列表或規則決定哪些圖片是 train / val
# 這裡簡單假設前 90% 為 train，後 10% 為 val
all_images = sorted([f for f in orig_dir.glob("*.png")])
random.seed(41)  # 任意整數都可以
random.shuffle(all_images)
num_train = int(len(all_images) * 0.9)

train_images = all_images[:num_train]


val_images = all_images[num_train:]

def convert_bbox_to_yolo(txt_path, img_path, save_path):
    """將 <class>,<x>,<y>,<w>,<h> 轉為 YOLO 格式"""
    from PIL import Image
    img = Image.open(img_path)
    w_img, h_img = img.size
    # img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

    yolo_lines = []
    with open(txt_path, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) != 5:
                continue
            cls, x, y, bw, bh = parts
            cls = int(cls)
            x, y, bw, bh = map(float, (x, y, bw, bh))

            # 計算中心點
            cx = x + bw / 2
            cy = y + bh / 2

            # 正規化
            cx /= w_img
            cy /= h_img
            bw /= w_img
            bh /= h_img

            yolo_lines.append(f"{cls} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")

    with open(save_path, "w") as f:
        f.write("\n".join(yolo_lines))

# 處理 train
for img_path in train_images:
    txt_path = orig_dir / (img_path.stem + ".txt")
    save_img_path = images_train_dir / img_path.name
    save_label_path = labels_train_dir / (img_path.stem + ".txt")

    copy2(img_path, save_img_path)
    convert_bbox_to_yolo(txt_path, img_path, save_label_path)

# 處理 val
for img_path in val_images:
    txt_path = orig_dir / (img_path.stem + ".txt")
    save_img_path = images_val_dir / img_path.name
    save_label_path = labels_val_dir / (img_path.stem + ".txt")

    copy2(img_path, save_img_path)
    convert_bbox_to_yolo(txt_path, img_path, save_label_path)

print("資料集轉換完成！")


資料集轉換完成！


# convert to patch

In [2]:
import os
import cv2

# === 參數設定 ===
W, H = 1920, 1080
PATCH_W, PATCH_H = 640, 540
ROWS, COLS = 2 , 3  # 共切成 2x3 = 6 塊
TVR = 0.9

In [3]:
def crop_and_save_patches(img, txt_pth, img_save_pth, img_name, bbox_save_pth):
  for r in range(ROWS):
    for c in range(COLS):
      x_start = c * PATCH_W
      y_start = r * PATCH_H
      x_end = min(x_start + PATCH_W, W)
      y_end = min(y_start + PATCH_H, H)

      # save and crop img
      patch_img = img[y_start:y_end, x_start:x_end] # np 
      patch_name = f"{img_name}_{r}_{c}.png"
      patch_img_path = os.path.join(img_save_pth, patch_name)

      # convert bbox
      patch_boxes = []
      with open(txt_pth, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) != 5:
                continue
            cls = int(parts[0])
            x, y, w, h = map(float, parts[1:5])   
            
            x1, y1 = x, y
            x2, y2 = x + w, y + h
            if x_start <= x1 < x_end and y_start <= y1 < y_end:
                x1 -= x_start
                y1 -= y_start
                x2 -= x_start
                y2 -= y_start

                # ensuring bbox in patch
                x2 = min(x2, PATCH_W)
                y2 = min(y2, PATCH_H)
                w = x2 - x1
                h = y2 - y1

                # to yolo
                xc = (x1 + w/2) / PATCH_W
                yc = (y1 + h/2) / PATCH_H
                w /= PATCH_W
                h /= PATCH_H
                patch_boxes.append([cls, xc, yc, w, h])

      if len(patch_boxes) > 0:
          # save
          cv2.imwrite(patch_img_path, patch_img)
          save_path = os.path.join(bbox_save_pth, f"{img_name}_{r}_{c}.txt")
          lines = [f"{cls} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}" for cls, cx, cy, bw, bh in patch_boxes]
          with open(save_path, "w") as f:
            f.write("\n".join(lines))

In [4]:
import random

# 建立資料夾
for d in [images_train_dir, images_val_dir, labels_train_dir, labels_val_dir]:
    d.mkdir(parents=True, exist_ok=True)

# 假設你有一個列表或規則決定哪些圖片是 train / val
# 這裡簡單假設前 90% 為 train，後 10% 為 val
all_images = sorted([f for f in orig_dir.glob("*.png")])
random.seed(42)  # 任意整數都可以
random.shuffle(all_images)
num_train = int(len(all_images) * TVR)

train_images = all_images[:num_train]
val_images = all_images[num_train:]

# 處理 train
for img_path in train_images:
    img = cv2.imread(str(img_path))
    txt_pth = orig_dir / (img_path.stem + ".txt")
    img_save_pth = images_train_dir
    img_name = img_path.stem
    bbox_save_pth = labels_train_dir
    
    crop_and_save_patches(img, txt_pth, img_save_pth, img_name, bbox_save_pth)

# 處理 val
for img_path in val_images:
    img = cv2.imread(str(img_path))
    txt_pth = orig_dir / (img_path.stem + ".txt")
    img_save_pth = images_val_dir
    img_name = img_path.stem
    bbox_save_pth = labels_val_dir

    crop_and_save_patches(img, txt_pth, img_save_pth, img_name, bbox_save_pth)

print("done!!")


done!!


# start training

In [3]:
yaml_path = "/kaggle/working/data.yaml"
with open(yaml_path, "w") as f:
    f.write(f"""
train: /kaggle/working/dataset/images/train
val: /kaggle/working/dataset/images/val
nc: 4
names: ['0', '1', '2', '3']
""")
print("data.yaml 已建立")

data.yaml 已建立


In [9]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

3732

In [10]:
import os
import numpy as np
import cv2
from ultralytics import YOLO

model = YOLO('yolov8s-p2.yaml')

# -----------------------------
# Stage 1: 訓練 feature extractor + classifier
# -----------------------------
model.train(
    data='/kaggle/working/data.yaml',      # 你的資料設定
    epochs=100,             # Stage 1 訓練輪數
    batch=4,
    imgsz=960,
    lr0=0.01,
    cos_lr=True,
    name= "exp11",
    # patience=20,
    optimizer='SGD',
    # ======= Data Augmentation 設定 =======
    momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0,
    warmup_momentum=0.8, warmup_bias_lr=0.1,
    close_mosaic=10,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    flipud=0.0, fliplr=0.5,
    degrees=0.0, translate=0.1, scale=0.5, shear=0.0,
)

Ultralytics 8.3.221 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-p2.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp112, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         95       3213      0.662      0.609      0.631      0.245
                     0         92       2258      0.857      0.914      0.932      0.421
                     1         56        140      0.732      0.771      0.789      0.305
                     2         52        425      0.447      0.193      0.242     0.0603
                     3         49        390       0.61      0.556      0.559      0.194
Speed: 0.5ms preprocess, 10.1ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to /kaggle/working/runs/detect/exp112


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e165e0bc650>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [11]:
import shutil

# 把模型檔壓縮成 zip
shutil.make_archive('/kaggle/working/exp11', 'zip',
                    
                    '/kaggle/working/runs/detect/exp112')

# 顯示路徑（可以在左邊檔案樹下載 best_model.zip）
print("✅ Model zipped at: /kaggle/working/best_model.zip")


✅ Model zipped at: /kaggle/working/best_model.zip


# cls re-weight

In [14]:
import os
import glob
import numpy as np
from collections import Counter

# === 設定 ===
label_dir = "dataset/labels/train"  # YOLO 格式的 label 檔
num_classes = 4                     # 修改為你的類別數
beta = 0.999                        # Effective Number 參數

# === 統計每個類別的樣本數 ===
cls_counter = Counter()
for label_file in glob.glob(os.path.join(label_dir, "*.txt")):
    with open(label_file, "r") as f:
        for line in f:
            if not line.strip():
                continue
            cls_id = int(line.split()[0])
            cls_counter[cls_id] += 1

counts = np.array([cls_counter[i] for i in range(num_classes)], dtype=np.float32)
counts[counts == 0] = 1e-6  # 防止除以零

# === Effective Number 權重 ===
effective_num = (1.0 - np.power(beta, counts)) / (1.0 - beta)
eff_weights = 1.0 / effective_num
eff_weights = eff_weights / eff_weights.sum() * num_classes  # normalize

# === 輸出結果 ===
weights_list = [round(float(w), 6) for w in eff_weights]
print(weights_list)


[0.890771, 1.27165, 0.94052, 0.897059]


In [15]:
import torch.nn as nn
import torch.nn.functional as F
from ultralytics.utils import loss

class WeightedFocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, logits, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        focal_term = (1 - pt) ** self.gamma
        if self.alpha is not None:
            BCE_loss *= self.alpha
        return (focal_term * BCE_loss).mean()

def custom_init(self, model):
    device = next(model.parameters()).device
    self.device = device
    self.nc = model.model[-1].nc
    self.hyp = model.args
    w = torch.tensor(weights_list, device=device)
    self.BCEcls = WeightedFocalLoss(alpha=w, gamma=2.0)
    self.BCEobj = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.0], device=device))
# 將自訂初始化替換進 YOLO
loss.FocalLoss.__init__ = custom_init


In [17]:
from ultralytics import YOLO

model = YOLO("/kaggle/working/runs/detect/exp112/weights/best.pt")

model.train(
    data="/kaggle/working/data.yaml",
    epochs=30,
    imgsz=640,
    batch=4,
    name="exp11_1",
    freeze = [0,1,2,3,4,5,6,7,8,9,10,11],
    lr0=0.01,
    optimizer='SGD',
    # cls=1.0,
    # ======= Data Augmentation 設定 =======
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    close_mosaic=10,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    flipud=0.0, fliplr=0.5,
    degrees=0.0, translate=0.1, scale=0.5, shear=0.0,
)

Ultralytics 8.3.221 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/working/runs/detect/exp112/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp11_12, nbs=64, nms=False, opset=None, optimize=False, optimizer=S

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         95       3213      0.606      0.551      0.556      0.209
                     0         92       2258      0.781      0.875      0.882      0.378
                     1         56        140       0.68      0.664      0.679      0.261
                     2         52        425       0.42      0.191      0.209     0.0508
                     3         49        390      0.543      0.474      0.456      0.146
Speed: 0.2ms preprocess, 5.0ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to /kaggle/working/runs/detect/exp11_12


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e165df02d90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [18]:
import shutil

# 把模型檔壓縮成 zip
shutil.make_archive('/kaggle/working/exp11_1', 'zip', '/kaggle/working/runs/detect/exp11_12')

# 顯示路徑（可以在左邊檔案樹下載 best_model.zip）
print("✅ Model zipped at: /kaggle/working/best_model.zip")


✅ Model zipped at: /kaggle/working/best_model.zip


# 一般推論

In [19]:
import os, csv
from ultralytics import YOLO

best_model = YOLO("/kaggle/working/runs/detect/exp11_12/weights/best.pt")
output_csv = "yolo11_pred_weight.csv"

TEST_FOLDER = "/kaggle/input/raw-dataset/CVPDL_hw2/CVPDL_hw2/test"
test_images = sorted([x for x in os.listdir(TEST_FOLDER) if x.lower().endswith(".png")])

with open(output_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Image_ID", "PredictionString"])

    for idx, img_name in enumerate(test_images, start=1):
        img_path = os.path.join(TEST_FOLDER, img_name)
        results = best_model(img_path, conf=0.10, augment=True, imgsz=960)

        det = results[0].boxes.xyxy.cpu().numpy()  # (N,4): x1,y1,x2,y2
        conf = results[0].boxes.conf.cpu().numpy()
        cls  = results[0].boxes.cls.cpu().numpy()  # 若你想保留類別

        parts = []
        if det.shape[0] == 0:
            # 無偵測到物件，填入 placeholder
            parts.append("0 0 0 0 0 0")
        else:
            for (x1, y1, x2, y2), c, cl in zip(det, conf, cls):
                w, h = x2 - x1, y2 - y1
                parts.append(f"{c:.4f} {int(x1)} {int(y1)} {int(w)} {int(h)} {int(cl)}")

        writer.writerow([idx, " ".join(parts)])

print("CSV 已完成:", output_csv)



image 1/1 /kaggle/input/raw-dataset/CVPDL_hw2/CVPDL_hw2/test/img0001.png: 544x960 63 0s, 6 1s, 91 2s, 129 3s, 62.2ms
Speed: 2.9ms preprocess, 62.2ms inference, 1.7ms postprocess per image at shape (1, 3, 544, 960)

image 1/1 /kaggle/input/raw-dataset/CVPDL_hw2/CVPDL_hw2/test/img0002.png: 544x960 188 0s, 28 1s, 1 2, 83 3s, 59.6ms
Speed: 2.7ms preprocess, 59.6ms inference, 1.8ms postprocess per image at shape (1, 3, 544, 960)

image 1/1 /kaggle/input/raw-dataset/CVPDL_hw2/CVPDL_hw2/test/img0003.png: 544x960 5 0s, 295 2s, 49.4ms
Speed: 3.0ms preprocess, 49.4ms inference, 1.3ms postprocess per image at shape (1, 3, 544, 960)

image 1/1 /kaggle/input/raw-dataset/CVPDL_hw2/CVPDL_hw2/test/img0004.png: 544x960 7 0s, 291 2s, 2 3s, 49.2ms
Speed: 3.0ms preprocess, 49.2ms inference, 1.5ms postprocess per image at shape (1, 3, 544, 960)

image 1/1 /kaggle/input/raw-dataset/CVPDL_hw2/CVPDL_hw2/test/img0005.png: 544x960 18 0s, 7 1s, 266 2s, 9 3s, 40.5ms
Speed: 3.0ms preprocess, 40.5ms inference, 1.3

# patch 推論

In [ ]:
import os, csv
from ultralytics import YOLO
import cv2

best_model = YOLO("/kaggle/working/runs/detect/exp10_13/weights/best.pt")
output_csv = "yolo11_pred_reweight10.csv"

TEST_FOLDER = "/kaggle/input/raw-dataset/CVPDL_hw2/CVPDL_hw2/test"
test_images = sorted([x for x in os.listdir(TEST_FOLDER) if x.lower().endswith(".png")])

with open(output_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Image_ID", "PredictionString"])

    for idx, img_name in enumerate(test_images, start=1):
      img_path = os.path.join(TEST_FOLDER, img_name)
      img = cv2.imread(img_path)
      parts = []
      for r in range(ROWS):
        for c in range(COLS):
          x_start = c * PATCH_W
          y_start = r * PATCH_H
          x_end = min(x_start + PATCH_W, W)
          y_end = min(y_start + PATCH_H, H)
          patch_img = img[y_start:y_end, x_start:x_end]

          results = best_model(patch_img, conf=0.001, augment=True, imgsz=640)
          det = results[0].boxes.xyxy.cpu().numpy()
          conf = results[0].boxes.conf.cpu().numpy()
          cls  = results[0].boxes.cls.cpu().numpy()

          # det reverse to full image
          if det.shape[0] == 0:
            # 無偵測到物件，填入 placeholder
            parts.append("0 0 0 0 0 0")
          else:
              for (x1, y1, x2, y2), c, cl in zip(det, conf, cls):
                x1 += x_start
                y1 += y_start
                x2 += x_start
                y2 += y_start
                x, y = x1, y1
                w = x2 - x1
                h = y2 - y1
                parts.append(f"{c:.4f} {int(x1)} {int(y1)} {int(w)} {int(h)} {int(cl)}")

      writer.writerow([idx, " ".join(parts)])

print("CSV 已完成:", output_csv)


0: 544x640 12 0s, 2 1s, 206 2s, 80 3s, 50.6ms
Speed: 1.7ms preprocess, 50.6ms inference, 2.2ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 102 0s, 1 1, 9 2s, 188 3s, 50.0ms
Speed: 1.3ms preprocess, 50.0ms inference, 1.6ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 97 0s, 17 1s, 47 2s, 60 3s, 49.9ms
Speed: 1.5ms preprocess, 49.9ms inference, 1.2ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 13 0s, 17 1s, 110 2s, 160 3s, 51.0ms
Speed: 1.5ms preprocess, 51.0ms inference, 1.3ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 43 0s, 2 1s, 16 2s, 12 3s, 38.8ms
Speed: 1.5ms preprocess, 38.8ms inference, 1.1ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 98 0s, 5 1s, 2 3s, 30.3ms
Speed: 1.3ms preprocess, 30.3ms inference, 1.1ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 203 0s, 14 1s, 32 2s, 51 3s, 30.0ms
Speed: 1.3ms preprocess, 30.0ms inference, 1.3ms postprocess per image at shape (1, 3, 544, 640)
